<h2>#Vamos começar importando a base de dados a ser estudada<h2>
#No nosso caso será um base do <b>ENEM 2022</b> e faremos alguns estudos sobre ela

Primeiro vamos importar a base

In [49]:
import pandas as pd

Leitura limitada em <b>1000</b> linhas

In [50]:
dfEnem = pd.read_csv('./dataset/MICRODADOS_ENEM_2022_REDUZIDO.csv', encoding= 'iso8859-1', sep = ';', nrows= 1000)

In [51]:
dfEnem.info() #informações da base

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 76 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   NU_INSCRICAO            1000 non-null   int64  
 1   NU_ANO                  1000 non-null   int64  
 2   TP_FAIXA_ETARIA         1000 non-null   int64  
 3   TP_SEXO                 1000 non-null   object 
 4   TP_ESTADO_CIVIL         1000 non-null   int64  
 5   TP_COR_RACA             1000 non-null   int64  
 6   TP_NACIONALIDADE        1000 non-null   int64  
 7   TP_ST_CONCLUSAO         1000 non-null   int64  
 8   TP_ANO_CONCLUIU         1000 non-null   int64  
 9   TP_ESCOLA               1000 non-null   int64  
 10  TP_ENSINO               239 non-null    float64
 11  IN_TREINEIRO            1000 non-null   int64  
 12  CO_MUNICIPIO_ESC        141 non-null    float64
 13  NO_MUNICIPIO_ESC        141 non-null    object 
 14  CO_UF_ESC               141 non-null    f

Aqui queremos filtrar um dataframe que tenha os participantes que responderam letra <b>"E"</b>*(Eu não concluí o Ensino Médio e nem estive matriculado(a) nesse período.)* na pergunta: *"Considerando a etapa de Ensino Médio, qual dessas situações está de acordo com o seu vínculo escolar durante a pandemia?"*

In [97]:
NaoCMed = dfEnem[dfEnem['Q001'] == 'E']

Aqui verificamos quantas linhas existem quando filtrado.

In [98]:
NaoCMed.shape

(238, 76)

Aqui verificamos quantos alunos tiveram a nota da REDAÇÃO *zerada*

In [53]:
notas_red_zero = NaoCMed[NaoCMed['NU_NOTA_REDACAO'] == 0]
notas_red_zero = notas_red_zero['NU_NOTA_REDACAO'].count()
notas_red_zero

8

Verificando quais tem valores *nulos* ou seja não fizeram a redação.

In [91]:
NaoCMed['NU_NOTA_REDACAO'].isnull().sum()

75

Aqui fazemos a importação da biblioteca "openai"(API), math (usada para interpretar valores nulos) e time(usado pra dar pausa entre as requisições).

In [101]:

import openai
import math 
import time

openai.api_key = "SUA_CHAVE_API" #Chave

def gerar_mensagem_motivacao(n_inscricao, nota_red): #Função que gera as mensagens, com dois parâmetros
  system_message = {"role": "system", "content": "Você é muito gentil e motivador, crie mensagens diferentes"} # mensagem inicial pro sistema
    
  if nota_red >= 600: #condicionais e mensagens para ser geradas
    user_message = {"role": "user", "content" : f"Crie uma mensagem motivacional para o inscrito do enem de numero {n_inscricao} que tirou uma nota alta ({nota_red}) na redação sabendo que a nota máxima é 1000 (se limite a 100 caracteres)(ENEM2022)"}
  elif math.isnan(nota_red): #se for nulo (não fez a redação)
    user_message = {"role": "user", "content" : f"Crie uma mensagem motivacional para o inscrito do enem de numero {n_inscricao} que não fez a redação (se limite a 100 caracteres)(ENEM2022)"}
  else:
    user_message = {"role": "user", "content" : f"Crie uma mensagem motivacional para o inscrito do enem de numero de inscrição {n_inscricao} que tirou uma nota baixa ({nota_red}) na redação (nota máxima é 1000) (se limite a 100 caracteres)(ENEM2022)"}
    
  messages = [system_message, user_message]
    
  completion = openai.ChatCompletion.create( #fazendo a requisição
      model="gpt-3.5-turbo",
      messages=messages
    )
  
  return completion.choices[0].message.content #mensagem retornada



In [102]:
msg_mot = NaoCMed[['NU_INSCRICAO','NU_NOTA_REDACAO']].iloc[1:20] # 19 linhas que foram geradas mensagens motivacionais
msg_mot['MSG_MOTIVACAO'] = msg_mot.apply(lambda row: (time.sleep(20),gerar_mensagem_motivacao(row['NU_INSCRICAO'], row['NU_NOTA_REDACAO']))[1], axis=1) #criando coluna das mensagens e chamando a função pra geração


In [103]:
msg_mot #Exibição do DataFrame

,NU_INSCRICAO,NU_NOTA_REDACAO,MSG_MOTIVACAO
5,210057850231,NaN,Não desista! Ainda há tempo para brilhar. #ENE...
7,210055778089,440.0,"""Você é capaz de alcançar resultados incríveis..."
9,210055461467,360.0,"""Não desista! A redação é apenas um desafio, v..."
10,210055665589,NaN,Você é capaz de superar qualquer desafio! Acre...
11,210054972570,940.0,Parabéns pelo seu desempenho incrível na redaç...
14,210054861834,NaN,Você é capaz de fazer a diferença! Não desista...
23,210057706656,760.0,Parabéns pelo excelente resultado na redação d...
27,210057728624,640.0,"Parabéns, 210057728624.0! Sua nota na redação ..."
30,210056235916,640.0,"Parabéns, campeão da redação! Sua nota é incrí..."
33,210057182714,760.0,"Parabéns, #210057182714! Sua pontuação na reda..."


In [104]:
msg_mot.to_excel('MotivacionalEnem.xlsx') #exporatando para excel